In [1]:
%%time

import malaya

CPU times: user 4.33 s, sys: 532 ms, total: 4.86 s
Wall time: 4.03 s


In [2]:
import tensorflow as tf

class _LANG_MODEL:
    def __init__(self, dimension = 32, output = 6):
        self.X = tf.sparse_placeholder(tf.int32, name = 'X_Placeholder')
        self.W = tf.sparse_placeholder(tf.int32, name = 'W_Placeholder')
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([400_000, dimension]))
        embed = tf.nn.embedding_lookup_sparse(
            embeddings, self.X, self.W, combiner = 'mean'
        )
        self.logits = tf.layers.dense(embed, output)
        self.logits = tf.identity(self.logits, name = 'logits')

In [3]:
model = _LANG_MODEL()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, '/Users/huseinzolkepli/Malaya/language-detection/deep/model.ckpt')

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/language-detection/deep/model.ckpt


In [4]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(400000, 32) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(32, 6) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(6,) dtype=float32_ref>]

In [5]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'out/model.ckpt')

'out/model.ckpt'

In [6]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['X_Placeholder/shape',
 'X_Placeholder/values',
 'X_Placeholder/indices',
 'W_Placeholder/shape',
 'W_Placeholder/values',
 'W_Placeholder/indices',
 'Placeholder',
 'Variable',
 'dense/kernel',
 'dense/bias',
 'logits']

In [7]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [8]:
freeze_graph('out', strings)

INFO:tensorflow:Restoring parameters from out/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 3 variables.
INFO:tensorflow:Converted 3 variables to const ops.
43 ops in the final graph.


In [9]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [10]:
g = load_graph('out/frozen_model.pb')

In [11]:
input_nodes = ['X_Placeholder/shape',
 'X_Placeholder/values',
 'X_Placeholder/indices',
 'W_Placeholder/shape',
 'W_Placeholder/values',
 'W_Placeholder/indices']
output_nodes = ['logits']

In [12]:
input_nodes = {label: g.get_tensor_by_name(f'import/{label}:0') for label in input_nodes}
output_nodes = {label: g.get_tensor_by_name(f'import/{label}:0') for label in output_nodes}

In [13]:
input_nodes, output_nodes

({'X_Placeholder/shape': <tf.Tensor 'import/X_Placeholder/shape:0' shape=(?,) dtype=int64>,
  'X_Placeholder/values': <tf.Tensor 'import/X_Placeholder/values:0' shape=(?,) dtype=int32>,
  'X_Placeholder/indices': <tf.Tensor 'import/X_Placeholder/indices:0' shape=(?, ?) dtype=int64>,
  'W_Placeholder/shape': <tf.Tensor 'import/W_Placeholder/shape:0' shape=(?,) dtype=int64>,
  'W_Placeholder/values': <tf.Tensor 'import/W_Placeholder/values:0' shape=(?,) dtype=int32>,
  'W_Placeholder/indices': <tf.Tensor 'import/W_Placeholder/indices:0' shape=(?, ?) dtype=int64>},
 {'logits': <tf.Tensor 'import/logits:0' shape=(?, 6) dtype=float32>})

In [14]:
test_sess = tf.Session(graph = g)

In [15]:
deep = malaya.language_detection.deep_model()
chinese_text = '今天是６月１８号，也是Muiriel的生日！'
from malaya.text.function import language_detection_textcleaning
strings = [language_detection_textcleaning(i) for i in [chinese_text]]
subs = [
    ' '.join(s)
    for s in deep._bpe.encode(strings, output_type = deep._type)
]
transformed = deep._vectorizer.transform(subs)

INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/language-detection/deep/model.ckpt


In [16]:
import numpy as np

coo = transformed.tocoo()
indices = np.array([coo.row, coo.col]).transpose()

In [17]:
%%time
o = test_sess.run(output_nodes['logits'],
        feed_dict = {input_nodes['X_Placeholder/shape']: coo.shape,
                    input_nodes['X_Placeholder/values']: coo.col,
                    input_nodes['X_Placeholder/indices']: indices,
                    input_nodes['W_Placeholder/shape']: coo.shape,
                    input_nodes['W_Placeholder/values']: coo.data,
                    input_nodes['W_Placeholder/indices']: indices})
o.shape

CPU times: user 678 ms, sys: 327 ms, total: 1 s
Wall time: 1.02 s


(1, 6)

In [18]:
from tensorflow.tools.graph_transforms import TransformGraph

In [19]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [20]:
input_nodes = ['X_Placeholder/shape',
 'X_Placeholder/values',
 'X_Placeholder/indices',
 'W_Placeholder/shape',
 'W_Placeholder/values',
 'W_Placeholder/indices']
output_nodes = ['logits']

In [21]:
pb = 'out/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
        
transformed_graph_def = TransformGraph(input_graph_def, 
                                       input_nodes,
                                       output_nodes, transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.
